# CSCI 3155 : Assignment 8

__Name__:

Qiuyang Fu

In [1]:
// TEST HELPER: Do not EDIT this cell please.
def passed(points: Int) {
    require(points >=0)
    if (points == 1) print(s"\n*** Tests Passed (1 point) ***\n")
    else print(s"\n*** Tests Passed ($points points) ***\n")
}

defined function passed

## Problem 1 : Type Checking (100 points)

Write a function `typecheck` that typechecks a Lettuce program.

$$\texttt{typecheck} : Program \rightarrow Type$$ 

This function should take as input a lettuce _Program_, and return what _Type_ it is. If it doesn't typecheck it should return a special value _TypeError_. Use the Lettuce typing notes on moodle for details on how typechecking lettuce works.

The tests for this assigment are hidden, so you will need to write your own tests! Note that you are encouraged to collaborate and share your test ideas on Piazza. To get full points, your implementation needs to match the specification from the Lettuce typing notes on moodle.

In [2]:
sealed trait Type
case object NumType extends Type
case object BoolType extends Type
case class FunType( ty_a : Type, ty_r : Type) extends Type
case object TypeError extends Type

sealed trait Expr
case class  Const(n : Int) extends Expr
case class  Ident(s : String) extends Expr
case object True extends Expr
case object False extends Expr
case class  Plus( e1 : Expr, e2 : Expr) extends Expr
case class  Minus( e1 : Expr, e2 : Expr) extends Expr
case class  Mult( e1 : Expr, e2 : Expr) extends Expr
case class  Neg( e1 : Expr) extends Expr
case class  And( e1 : Expr, e2 : Expr) extends Expr
case class  Or( e1 : Expr, e2 : Expr) extends Expr
case class  Geq( e1 : Expr, e2 : Expr) extends Expr
case class  Eq( e1 : Expr, e2 : Expr) extends Expr
case class  ITE( ec : Expr, et : Expr, ef : Expr) extends Expr
case class  Let( name : String, ty_d : Type, ed : Expr, ei : Expr) extends Expr
case class  FunDef( arg : String, ty_a : Type, eb : Expr) extends Expr
case class  FunCall( ef : Expr, ea : Expr) extends Expr
case class  LetRec( f : String, ty_f : Type, x : String, ty_x : Type, eb : Expr, ei : Expr) extends Expr

sealed trait Program
case class Main(p : Expr) extends Program

defined trait Type
defined object NumType
defined object BoolType
defined class FunType
defined object TypeError
defined trait Expr
defined class Const
defined class Ident
defined object True
defined object False
defined class Plus
defined class Minus
defined class Mult
defined class Neg
defined class And
defined class Or
defined class Geq
defined class Eq
defined class ITE
defined class Let
defined class FunDef
defined class FunCall
defined class LetRec
defined trait Program
defined class Main

In [3]:
def typecheck(p : Program) = p match {
    case Main(e: Expr) => typecheckExpr(e, Map())
    case _ => TypeError
}

def typecheckExpr(e: Expr, env: Map[String, Type]) : Type = {
    def typeEquals(e1: Expr, t1: Type, e2: Expr, t2: Type, resultType: Type) = {
        val e1Type = typecheckExpr(e1, env)
        val e2Type = typecheckExpr(e2, env)
        if (e1Type == t1 && e2Type == t2) {
            resultType
        } else {
            TypeError
        }
    }
    e match {
        case Const(_)  => NumType
        case True => BoolType
        case False => BoolType
        case Ident(x: String) => env.getOrElse(x, TypeError)
        case Plus(e1: Expr, e2: Expr) => typeEquals(e1, NumType, e2, NumType, NumType)
        case Minus(e1: Expr, e2: Expr) => typeEquals(e1, NumType, e2, NumType, NumType)
        case Mult(e1: Expr, e2: Expr) => typeEquals(e1, NumType, e2, NumType, NumType)
        case And(e1: Expr, e2: Expr) => typeEquals(e1, BoolType, e2, BoolType, BoolType)
        case Or(e1: Expr, e2: Expr) => typeEquals(e1, BoolType, e2, BoolType, BoolType)
        case Neg(e: Expr) => typecheckExpr(e, env) match {
            case BoolType => BoolType
            case NumType => NumType
            case _ => TypeError
        }
        case Geq(e1: Expr, e2: Expr) => typeEquals(e1, NumType, e2, NumType, BoolType)
        case Eq(e1: Expr, e2: Expr) => (typecheckExpr(e1, env), typecheckExpr(e2, env)) match {
            case (TypeError, _) => TypeError
            case (_, TypeError) => TypeError
            case _ => BoolType
        }
        case ITE( ec : Expr, et : Expr, ef : Expr) => (typecheckExpr(ec, env), typecheckExpr(et, env), typecheckExpr(ef, env)) match {
            case (_, _, TypeError) => TypeError
            case (_, TypeError, _) => TypeError
            case (BoolType, t1, t2) => {
                if (t1 == t2) {
                    t1
                } else {
                    TypeError
                }
            }
            case _ => TypeError
        }
        case Let( name : String, ty_d : Type, ed : Expr, ei : Expr) => {
            if (typecheckExpr(ed, env) == ty_d){
                val new_env = env + (name -> ty_d)
                typecheckExpr(ei, new_env)
            } else {
                TypeError
            }
        }
        case FunDef( arg : String, ty_a : Type, eb : Expr) => {
            val new_env = env + (arg -> ty_a)
            val new_t = typecheckExpr(eb, new_env)
            if (new_t == TypeError) {
                TypeError
            } else {
                FunType(ty_a, new_t)
            }
        }
        case FunCall( ef : Expr, ea : Expr) =>  typecheckExpr(ef, env) match {
            case FunType(ty_a, ty_r) => {
                if (typecheckExpr(ea, env) == ty_a){
                    ty_r
                } else {
                    TypeError
                }
            }
            case _ => TypeError
        }
        case LetRec( f : String, ty_f : Type, x : String, ty_x : Type, eb : Expr, ei : Expr) => {
            val new_env = env + (x -> ty_f, f -> FunType(ty_f, ty_x))
            val new_t = typecheckExpr(eb, new_env)
            if (new_t == ty_x){
                typecheckExpr(ei, env + (f -> FunType(ty_f, ty_x)))
            } else {
                TypeError
            }
        }
        case _ => TypeError
    }
}

defined function typecheck
defined function typecheckExpr

In [4]:
val p1 : Program = Main( Const(10) )
assert( typecheck(p1) == NumType, "test 1")
passed(20)
println("Hidden Tests : up to 80 points remaining")


*** Tests Passed (20 points) ***
Hidden Tests : up to 80 points remaining


p1: Program = Main(Const(10))

In [5]:
// test Const 
val p1 = Main( True )
val p2 = Main( False )
assert( typecheck(p1) == BoolType, "test 1")
assert( typecheck(p2) == BoolType, "test 2")

p1: Main = Main(True)
p2: Main = Main(False)

In [6]:
// test Plus
val p1 = Main( Plus(Const(1), Const(2)) )
val p2 = Main( Plus(Const(1), False) )
assert( typecheck(p1) == NumType, "test 1")
assert( typecheck(p2) == TypeError, "test 2")

p1: Main = Main(Plus(Const(1), Const(2)))
p2: Main = Main(Plus(Const(1), False))

In [7]:
// test Minus
val p1 = Main( Minus(Const(2), Const(1)) )
val p2 = Main( Minus(Const(2), False) )
assert( typecheck(p1) == NumType, "test 1")
assert( typecheck(p2) == TypeError, "test 2")

p1: Main = Main(Minus(Const(2), Const(1)))
p2: Main = Main(Minus(Const(2), False))

In [8]:
// test And
val p1 = Main( And(True, True) )
val p2 = Main( And(Const(2), False) )
assert( typecheck(p1) == BoolType, "test 1")
assert( typecheck(p2) == TypeError, "test 2")

p1: Main = Main(And(True, True))
p2: Main = Main(And(Const(2), False))

In [9]:
// test Or
val p1 = Main( Or(True, True) )
val p2 = Main( Or(Const(2), False) )
assert( typecheck(p1) == BoolType, "test 1")
assert( typecheck(p2) == TypeError, "test 2")

p1: Main = Main(Or(True, True))
p2: Main = Main(Or(Const(2), False))

In [10]:
// test Neg 
val p1 = Main( Neg(True) )
val p2 = Main( Neg(And(True, True)) )
assert( typecheck(p1) == BoolType, "test 1")
assert( typecheck(p2) == BoolType, "test 2")

p1: Main = Main(Neg(True))
p2: Main = Main(Neg(And(True, True)))

In [11]:
// test Neg
val p1 = Main( Neg(And(True, Or(True, False))) )
val p2 = Main( Neg(And(True, Const(10))) )
val p3 = Main( Neg(Const(1)) )
val p4 = Main( Neg(Minus(Const(1), Const(2))) )
val p5 = Main( Neg(Minus(Const(1), False)) )
assert( typecheck(p1) == BoolType, "test 1")
assert( typecheck(p2) == TypeError, "test 2")
assert( typecheck(p3) == NumType, "test 3")
assert( typecheck(p4) == NumType, "test 4")
assert( typecheck(p5) == TypeError, "test 5")

p1: Main = Main(Neg(And(True, Or(True, False))))
p2: Main = Main(Neg(And(True, Const(10))))
p3: Main = Main(Neg(Const(1)))
p4: Main = Main(Neg(Minus(Const(1), Const(2))))
p5: Main = Main(Neg(Minus(Const(1), False)))

In [12]:
// test Geq
val p1 = Main( Geq(Const(1), Const(2)) )
val p2 = Main( Geq(Minus(Const(2), Const(1)), Const(2)) )
val p3 = Main( Geq(And(True, False),  Const(2)) )
val p4 = Main( Geq(Plus(True, Const(3)),  Const(2)) )
val p5 = Main( Geq(Const(2), Plus(True, Const(3))) )
val p6 = Main( Geq(False, False) )
assert( typecheck(p1) == BoolType, "test 1")
assert( typecheck(p2) == BoolType, "test 2")
assert( typecheck(p3) == TypeError, "test 3")
assert( typecheck(p4) == TypeError, "test 4")
assert( typecheck(p5) == TypeError, "test 5")
assert( typecheck(p6) == TypeError, "test 6")

p1: Main = Main(Geq(Const(1), Const(2)))
p2: Main = Main(Geq(Minus(Const(2), Const(1)), Const(2)))
p3: Main = Main(Geq(And(True, False), Const(2)))
p4: Main = Main(Geq(Plus(True, Const(3)), Const(2)))
p5: Main = Main(Geq(Const(2), Plus(True, Const(3))))
p6: Main = Main(Geq(False, False))

In [13]:
// test Eq
val p1 = Main( Eq(Const(1), Const(2)) )
val p2 = Main( Eq(Minus(Const(2), Const(1)), Const(2)) )
val p3 = Main( Eq(And(True, False),  Const(2)) )
val p4 = Main( Eq(Plus(True, Const(3)),  Const(2)) )
val p5 = Main( Eq(Const(2), Plus(True, Const(3))) )
val p6 = Main( Eq(False, False) )
assert( typecheck(p1) == BoolType, "test 1")
assert( typecheck(p2) == BoolType, "test 2")
assert( typecheck(p3) == BoolType, "test 3")
assert( typecheck(p4) == TypeError, "test 4")
assert( typecheck(p5) == TypeError, "test 5")
assert( typecheck(p6) == BoolType, "test 6")

p1: Main = Main(Eq(Const(1), Const(2)))
p2: Main = Main(Eq(Minus(Const(2), Const(1)), Const(2)))
p3: Main = Main(Eq(And(True, False), Const(2)))
p4: Main = Main(Eq(Plus(True, Const(3)), Const(2)))
p5: Main = Main(Eq(Const(2), Plus(True, Const(3))))
p6: Main = Main(Eq(False, False))

In [14]:
// test ITE
val p1 = Main( ITE(False, Const(2), Const(1)) )
val p2 = Main( ITE(False, True, False) )
val p5 = Main( ITE(False, Const(2), False))
assert( typecheck(p1) == NumType, "test 1")
assert( typecheck(p2) == BoolType, "test 2")
assert( typecheck(p5) == TypeError, "test 5")

p1: Main = Main(ITE(False, Const(2), Const(1)))
p2: Main = Main(ITE(False, True, False))
p5: Main = Main(ITE(False, Const(2), False))

In [15]:
// test FunDef
val p1 = Main( FunDef( "x", NumType, Plus(Ident("x"), Const(1))) )
assert( typecheck(p1) == FunType(NumType, NumType), "test 1")
val p2 = Main( FunDef( "x", NumType, Eq(Ident("x"), Const(1))) )
assert( typecheck(p2) == FunType(NumType, BoolType), "test 2")

p1: Main = Main(FunDef("x", NumType, Plus(Ident("x"), Const(1))))
p2: Main = Main(FunDef("x", NumType, Eq(Ident("x"), Const(1))))

In [16]:
// test Let
val p1 = Main( FunDef("g", NumType, Let("h", NumType, Const(1), Mult(Ident("g"), Ident("h")))) )
assert( typecheck(p1) == FunType(NumType, NumType), "test 1")

p1: Main = Main(
  FunDef(
    "g",
    NumType,
    Let("h", NumType, Const(1), Mult(Ident("g"), Ident("h")))
  )
)

In [17]:
// test FunCall
val p1 = Main( FunCall(FunDef("a", NumType, Const(1)), Const(1)) )
assert( typecheck(p1) == NumType, "test 1")
val p2 = Main( FunCall(FunDef("b", NumType, Const(1)), Ident("b")) )
assert( typecheck(p2) == TypeError, "test 2")
val p3 = Main( FunCall(FunDef("c", NumType, Ident("d")), Const(1)) )
assert( typecheck(p3) == TypeError, "test 3")
val p4 = Main( FunCall(FunDef("a", NumType, Eq(Ident("a"), Const(1))), Const(1)) )
assert( typecheck(p4) == BoolType, "test 4")

p1: Main = Main(FunCall(FunDef("a", NumType, Const(1)), Const(1)))
p2: Main = Main(FunCall(FunDef("b", NumType, Const(1)), Ident("b")))
p3: Main = Main(FunCall(FunDef("c", NumType, Ident("d")), Const(1)))
p4: Main = Main(
  FunCall(FunDef("a", NumType, Eq(Ident("a"), Const(1))), Const(1))
)

In [18]:
// test LetRec
val p1 = Main( LetRec("f", NumType, "x", NumType,
            Plus(FunCall(Ident("f"), Plus(Ident("x"), Const(1))), Const(2)), 
            FunCall(Ident("f"), Const(2))) )
assert( typecheck(p1) == NumType, "test 1")

p1: Main = Main(
  LetRec(
    "f",
    NumType,
    "x",
    NumType,
    Plus(FunCall(Ident("f"), Plus(Ident("x"), Const(1))), Const(2)),
    FunCall(Ident("f"), Const(2))
  )
)

In [19]:
println("Hidden test")

Hidden test


In [20]:
println("Hidden test")

Hidden test


In [21]:
println("Hidden test")

Hidden test


In [22]:
println("Hidden test")

Hidden test


In [23]:
println("Hidden test")

Hidden test


In [24]:
println("Hidden test")

Hidden test


In [25]:
println("Hidden test")

Hidden test


In [26]:
println("Hidden test")

Hidden test


In [27]:
println("Hidden test")

Hidden test


In [28]:
println("Hidden test")

Hidden test


In [29]:
println("Hidden test")

Hidden test


In [30]:
println("Hidden test")

Hidden test


In [31]:
println("Hidden test")

Hidden test


In [32]:
println("Hidden test")

Hidden test


In [33]:
println("Hidden test")

Hidden test


In [34]:
println("Hidden test")

Hidden test


In [35]:
println("Hidden test")

Hidden test


In [36]:
println("Hidden test")

Hidden test


In [37]:
println("Hidden test")

Hidden test


In [38]:
println("Hidden test")

Hidden test


In [39]:
println("Hidden test")

Hidden test


In [40]:
println("Hidden test")

Hidden test


In [41]:
println("Hidden test")

Hidden test


In [42]:
println("Hidden test")

Hidden test


In [43]:
println("Hidden test")

Hidden test


In [44]:
println("Hidden test")

Hidden test


In [45]:
println("Hidden test")

Hidden test


In [46]:
println("Hidden test")

Hidden test


In [47]:
println("Hidden test")

Hidden test


In [48]:
println("Hidden test")

Hidden test


In [49]:
println("Hidden test")

Hidden test


In [50]:
println("Hidden test")

Hidden test


In [51]:
println("Hidden test")

Hidden test
